In [ ]:
import os
import warnings
import numpy as np
import healpy as hp
import bokeh
import colorcet as cc
import pandas as pd
import panel as pn
from astropy.time import Time
import sqlite3
import urllib.request
import astropy.coordinates

import healsparse as hsp

from uranography.api import (
    Planisphere,
    ArmillarySphere,
    MollweideMap,
    HorizonMap,
    split_healpix_by_resolution,
    load_bright_stars,
    CameraFootprintPerimeter,
)

In [ ]:
pn.extension()

In [ ]:
night = Time("2026-05-30", location=astropy.coordinates.EarthLocation.of_site("Cerro Pachon"))

In [ ]:
warnings.filterwarnings('ignore', message=r'.*Tried to get polar motions for times after IERS data is valid.*')

# Sample data

Just to have some data to plot, make a simple data source of a few globular clusters, just as you would for any other `bokeh` plot:

In [ ]:
sample_data_source = bokeh.models.ColumnDataSource({
    'name': ['47 Tuc', 'M79', 'M68', 'M53', 'omega Cen', 'M3', 'M5'],
    'coords': [(6.024, -72.081), (81.046, -24.525), (189.867, -26.744), (198.23, 18.168), (201.697, -47.48), (205.548, 28.377), (229.638, 2.081)]
})

# pandas DataFrames are prettier in jupyter, so us to_df() to show it:
sample_data_source.to_df()

# Simplest example

In [ ]:
psphere = Planisphere()
psphere.decorate()

# At this point, psphere.plot is a normal instance of bokeh.plotting.Figure, and you can use the standard bokeh API to add whatever you want to this basic figure.
# To use projected coordinates, use the psphere.x_transform and psphere.y_transform methods.
psphere.plot.circle(x=psphere.x_transform('coords'), y=psphere.y_transform('coords'), color='red', source=sample_data_source)

psphere.notebook_display()

# Slightly less simple example

Similar to the example above, but taking more control of the details and including an `mjd` slider:

In [ ]:
plot = bokeh.plotting.figure(
    frame_width=512,
    frame_height=512,
    match_aspect=True,
    title="Sample 1",
)
psphere = Planisphere(mjd=night.mjd, plot=plot)
psphere.add_mjd_slider()
psphere.add_graticules(
    graticule_kwargs={
        "min_decl": -80,
        "max_decl": 80,
        "decl_space": 20,
        "min_ra": 0,
        "max_ra": 360,
        "ra_space": 30,
    },
    line_kwargs={"color": "lightgray"},
)
psphere.add_ecliptic(color="green")
psphere.add_galactic_plane(color="blue")
psphere.add_horizon()
psphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2})

# Plot our sample data using the standard bokeh API
psphere.plot.star(x=psphere.x_transform('coords'), y=psphere.y_transform('coords'), color='orange', size=8, source=sample_data_source)

psphere.notebook_display()

# Simple armillary sphere example

In [ ]:
plot = bokeh.plotting.figure(
    frame_width=512,
    frame_height=512,
    match_aspect=True,
    title="Sample 2",
)
asphere = ArmillarySphere(mjd=night.mjd, plot=plot)
asphere.add_mjd_slider()
asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
asphere.add_horizon()
asphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2})

# Plot our sample data using the standard bokeh API
asphere.plot.circle(x=asphere.x_transform('coords'), y=asphere.y_transform('coords'), color='orange', size=8, source=sample_data_source)
asphere.connect_controls(sample_data_source) ;# Needed to update the projected x and y when the sliders move.

asphere.notebook_display()

# Multiple connected views

In [ ]:
data_source = {}

asphere_plot = bokeh.plotting.figure(
    frame_width=512,
    frame_height=512,
    match_aspect=True,
    title="Sample 3a",
)
asphere = ArmillarySphere(mjd=night.mjd, plot=asphere_plot)
asphere.add_mjd_slider()

asphere.plot.star(x=asphere.x_transform('coords'), y=asphere.y_transform('coords'), color='orange', size=8, source=sample_data_source)
asphere.connect_controls(sample_data_source)

asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
data_source["horizon"] = asphere.add_horizon()
data_source["high_X"] = asphere.add_horizon(
    zd=70, line_kwargs={"color": "red", "line_width": 2}
)

psphere_plot = bokeh.plotting.figure(
    frame_width=512,
    frame_height=512,
    match_aspect=True,
    title="Sample 3b",
)
psphere = Planisphere(mjd=night.mjd, plot=psphere_plot)
psphere.sliders['mjd'] = asphere.sliders['mjd']
psphere.plot.star(x=psphere.x_transform('coords'), y=psphere.y_transform('coords'), color='orange', size=8, source=sample_data_source)
psphere.add_graticules(
    graticule_kwargs={
        "min_decl": -80,
        "max_decl": 80,
        "decl_space": 20,
        "min_ra": 0,
        "max_ra": 360,
        "ra_space": 30,
    },
    line_kwargs={"color": "lightgray"},
)
psphere.add_ecliptic()
psphere.add_galactic_plane()
psphere.add_horizon(data_source=data_source["horizon"])
psphere.add_horizon(
    data_source=data_source["high_X"], line_kwargs={"color": "red", "line_width": 2}
)

msphere_plot = bokeh.plotting.figure(
    frame_width=1024,
    frame_height=512,
    match_aspect=False,
    title="Sample 3c",
)
msphere = MollweideMap(mjd=night.mjd, plot=msphere_plot)
msphere.sliders['mjd'] = asphere.sliders['mjd']
msphere.plot.star(x=msphere.x_transform('coords'), y=msphere.y_transform('coords'), color='orange', size=8, source=sample_data_source)

msphere.add_graticules(
    graticule_kwargs={
        "min_decl": -80,
        "max_decl": 80,
        "decl_space": 20,
        "min_ra": 0,
        "max_ra": 360,
        "ra_space": 30,
    },
    line_kwargs={"color": "lightgray"},
)

msphere.add_ecliptic()
msphere.add_galactic_plane()
msphere.add_horizon(data_source=data_source["horizon"])
msphere.add_horizon(
    data_source=data_source["high_X"], line_kwargs={"color": "red", "line_width": 2}
)

viewable = pn.Column(msphere.figure, pn.Row(asphere.figure, psphere.figure))

# These would be necessary if we had healpix maps
msphere.update(viewable)
asphere.update(viewable)
psphere.update(viewable)
pn.io.push_notebook(viewable)

viewable

# Add the sun, moon, and stars

Use `astropy` to get the position of the sun and moon:

In [ ]:
sun_coords = astropy.coordinates.get_sun(night)
moon_coords = astropy.coordinates.get_moon(night)
sun_coords, moon_coords

Load the Yale bright star catalog.

If we don't have it locally, get it:

In [ ]:
stars_fname = "bsc5.dat.gz"
if not os.path.isfile(stars_fname):
    urllib.request.urlretrieve("http://tdc-www.harvard.edu/catalogs/bsc5.dat.gz", stars_fname)

Now, load the stars:

In [ ]:
stars = load_bright_stars(stars_fname)
len(stars)

This is way too many stars. Only consider the brightest:

In [ ]:
stars.query("Vmag<3.5", inplace=True)
len(stars)

Now show the figure:

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_mjd_slider()
asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
asphere.add_horizon()
asphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2})

asphere.add_marker(
    sun_coords.ra.deg,
    sun_coords.dec.deg,
    name="Sun",
    glyph_size=15,
    circle_kwargs={"color": "brown"},
)
asphere.add_marker(
    moon_coords.ra.deg,
    moon_coords.dec.deg,
    name="Moon",
    glyph_size=15,
    circle_kwargs={"color": "orange"},
)

# Scale the size of the star markers with the magnitude of the stars
stars["glyph_size"] = 15 * (1.01 - stars["Vmag"] / stars["Vmag"].max())

# Actually add the stars
asphere.add_stars(stars, mag_limit_slider=True, star_kwargs={"color": "black"})

# Set the limit of the slider according to the stars we've included
asphere.sliders["mag_limit"].end = stars["Vmag"].max()

asphere.notebook_display()

# Horizon coordinates

You can show a map in a horizon (az/alt polar) projection:

In [ ]:
hsphere = HorizonMap(mjd=night.mjd)
hsphere.add_mjd_slider()
hsphere.add_horizon_graticules()
hsphere.add_horizon()
hsphere.add_ecliptic()
hsphere.add_galactic_plane()

hsphere.add_marker(
    sun_coords.ra.deg,
    sun_coords.dec.deg,
    name="Sun",
    glyph_size=15,
    circle_kwargs={"color": "brown"},
)
hsphere.add_marker(
    moon_coords.ra.deg,
    moon_coords.dec.deg,
    name="Moon",
    glyph_size=15,
    circle_kwargs={"color": "orange"},
)
hsphere.add_stars(stars, mag_limit_slider=True, star_kwargs={"color": "black"})
hsphere.notebook_display()

You can show horizon graticules in any of the projections:

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_mjd_slider()
asphere.add_graticules()
asphere.add_ecliptic()
asphere.add_galactic_plane()
asphere.add_horizon_graticules(line_kwargs={"color": "red", "line_dash": "dotted"})
asphere.notebook_display()

# Show a healpix map

Get the healpix dust map from `rubin_sim` as an example healpix map.

In [ ]:
depth_fname = 'r_depth.fits'

In [ ]:
depth = hp.ud_grade(hp.read_map(depth_fname), 32)

Make a `bokeh` color map:

In [ ]:
cmap = bokeh.transform.linear_cmap(
    "value", "Inferno256", 23, 27.5
)

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_healpix(depth, nside=32, cmap=cmap)
asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen")
asphere.add_galactic_plane(color="lightblue")
asphere.notebook_display()

# Show a healsparse map

At nsides greater than 32, interactivity can be sluggish. Sometimes you can reduce the number of pixels by only displaying some regions of the sky. For example, we can show only the high dust areas in the dust map above:

In [ ]:
# Get a higher resolution healpix map
depth64_ring = hp.ud_grade(hp.read_map(depth_fname), 64)
depth64 = hp.reorder(depth64_ring, inp="RING", out="NESTED")

# Cut off any pixels lower than the bottom for our color map
depth64[depth64 < cmap["transform"].low] = hp.UNSEEN

# Make a healsparse map with just the seen healpixel
depth_hsp = hsp.HealSparseMap(nside_coverage=16, healpix_map=depth64)

Make a color map such that low dust areas are near white, and thus fall to white as dust drops:

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
asphere.add_healpix(depth_hsp, nside=64, cmap=cmap)
asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen")
asphere.add_galactic_plane(color="lightblue")
asphere.notebook_display()

# Survey footprint

Get a survey footprint (in heapix format):

In [ ]:
nside = 64
footprint = hp.ud_grade(hp.read_map('sample_survey_footprint.fits'), nside)

In [ ]:
low_nside = 16
this_footprint = footprint.copy()
this_footprint[this_footprint == 0] = hp.UNSEEN
footprint_high, footprint_low = split_healpix_by_resolution(
    this_footprint, low_nside, nside
)

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)

cmap = bokeh.transform.linear_cmap("value", cc.palette["rainbow4"], 0, 1)

asphere.add_healpix(footprint_high, nside=nside, cmap=cmap)
asphere.add_healpix(footprint_low, nside=low_nside, cmap=cmap)
asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen")
asphere.add_galactic_plane(color="lightblue")
asphere.notebook_display()

# Arbitrary bokeh

## Data sources with point locations

Use `asphere.make_points` to create the data source.

The `plot` member of `SphereMap` and its children (`ArmillarySphere`, etc.) is just a normal `bokeh.plotting.Figure`.

The `proj_transform`, `x_transform`, and `y_transform` methods of `SphereMap` and its children create `bokeh` transforms that can be used to handle projections.

So, to plot on the projection plane using any of the varous methods of `bokeh.plotting.Figure`, create an appropriate `bokeh` data source and use the relevant `transform` method of the `SphereMap` instance to provide the transforms.

By default, when you create your own `bokeh` data source, the transforms will not be updated when you manipulate the plot with the controls. To make your new points reactive, you also need to call the `connect_controls` method of your `SphereMap` instance on your `bokeh` data source.

There are two ways to apply the transforms. The first method requires creation of a data source with a column with the coordinates expressed as R.A., declination tuples (expressed as degrees):

In [ ]:
npoints = 100

asphere = ArmillarySphere(mjd=night.mjd)

ra_values = np.random.random(npoints)*360
decl_values = np.random.random(npoints)*180-90

# Create a pandas.DataFrame from wich
sample_data_source = bokeh.models.ColumnDataSource(
    {
        "name": [f"sample_{i+1}" for i in np.arange(npoints)],
        "foo": 2+10*np.random.random(npoints), # Just some random values
        "coords": [(ra_values[i], decl_values[i]) for i in range(npoints)]
    }
)

asphere = ArmillarySphere(mjd=night.mjd)
asphere.connect_controls(sample_data_source)

asphere.plot.asterisk(asphere.x_transform('coords'), asphere.y_transform('coords'), size="foo", source=sample_data_source)
asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen")
asphere.add_galactic_plane(color="lightblue")
asphere.notebook_display()

Alternately, you can supply `ra` and `decl` columns in the data source with R.A. and declination in degrees, and use the `proj_transform` method instead.

This approch is less flexible and less natural to the `bokeh` API, but it works, and can be more convenient:

In [ ]:
npoints = 100
sample_df = pd.DataFrame(
    {
        "name": "sample",
        "foo": 2+10*np.random.random(npoints), # Just some random values
        "ra": np.random.random(npoints) * 360,
        "decl": np.random.random(npoints) * 180 - 90,
    }
)
asphere = ArmillarySphere(mjd=night.mjd)
sample_data_source = bokeh.models.ColumnDataSource(sample_df)
asphere.connect_controls(sample_data_source)

asphere.plot.asterisk(asphere.proj_transform('x', sample_data_source), asphere.proj_transform('y', sample_data_source), size="foo", source=sample_data_source)
asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen")
asphere.add_galactic_plane(color="lightblue")
asphere.notebook_display()

## Plotting lines and shapes
Plotting lines, patches, and polygons follows the same bokeh api.

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)

patch_data_source = bokeh.models.ColumnDataSource({
    'coords': [(100, -45), (100, -50), (115, -50), (115, -45)]
})
asphere.plot.patch(asphere.x_transform('coords'), asphere.y_transform('coords'), color='red', source=patch_data_source)
asphere.connect_controls(patch_data_source)

line_data_source = bokeh.models.ColumnDataSource({
    'coords': [(145, -38), (145, -42), (155, -38), (155, -30)]
})
asphere.plot.line(asphere.x_transform('coords'), asphere.y_transform('coords'), color='blue', width=5, source=line_data_source)
asphere.connect_controls(line_data_source)

asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen")
asphere.add_galactic_plane(color="lightblue")
asphere.notebook_display()

## Plotting the same shape with different centers

Use `asphere.make_patches_data_source` to create the data source.

In [ ]:
npoints = 25
sample_df = pd.DataFrame(
    {
        "center_ra": np.random.random(npoints) * 360,
        "center_decl": np.random.random(npoints) * 180 - 90,
        "rotation": 180 * np.random.random(npoints),
    }
)

camera_perimeter = CameraFootprintPerimeter('rubin_idealized.txt')
ras, decls = camera_perimeter(
    sample_df.center_ra, sample_df.center_decl, sample_df.rotation
)
sample_df["ra"] = ras
sample_df["decl"] = decls
sample_df.head()

In [ ]:
asphere = ArmillarySphere(mjd=night.mjd)
sample_data_source = bokeh.models.ColumnDataSource(sample_df)
asphere.connect_controls(sample_data_source)
asphere.plot.patches(
    xs=asphere.proj_transform('x', sample_data_source),
    ys=asphere.proj_transform('y', sample_data_source),
    source=sample_data_source,
    line_color="blue",
    fill_color="red",
    fill_alpha=0.2,
)
asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen")
asphere.add_galactic_plane(color="lightblue")
asphere.notebook_display()

# Plotting visits for a night, with bells and whistles

Start with a text file with the list of exposures we want to plot:

In [ ]:
!head exposures.txt

Read it, and add the camera perimeter:

In [ ]:
visits = pd.read_csv('exposures.txt', delim_whitespace=True, header=0, index_col=False)
    
visits["ra"], visits["decl"] = camera_perimeter(
    visits.boresight_ra, visits.boresight_decl, visits.rotation
)
visits

In [ ]:
initial_mjd = visits.mjd.max()

In [ ]:
# Use the smallest categorical bokeh palette for the number of bands actually used
used_bands = [b for b in "ugrizy" if b in visits['band'].values]
num_bands = len(used_bands)
base_palette = bokeh.palettes.Spectral
try:
    band_palette = dict(zip(used_bands, base_palette[num_bands]))
except KeyError:
    band_palette = dict(zip(used_bands, base_palette[3][:num_bands]))

visits["color"] = visits.band.map(band_palette)

Sun and moon coordinates:

In [ ]:
sun_coords = astropy.coordinates.get_sun(night)
moon_coords = astropy.coordinates.get_moon(night)

Include the survey footprint:

In [ ]:
nside = 32
footprint = hp.ud_grade(hp.read_map('sample_survey_footprint.fits'), nside)

low_nside = 16
this_footprint = footprint.copy()
this_footprint[this_footprint == 0] = hp.UNSEEN
footprint_high, footprint_low = split_healpix_by_resolution(
    this_footprint, low_nside, nside
)

In [ ]:
data_source = {}

asphere_plot = bokeh.plotting.figure(
    frame_width=768,
    frame_height=512,
    match_aspect=True,
    title="Armillary Sphere",
)

asphere = ArmillarySphere(mjd=initial_mjd, plot=asphere_plot)

Add the footprint in the background:

In [ ]:
# Faint gray footprint in the background
cmap = bokeh.transform.linear_cmap(
    "value", list(reversed(bokeh.palettes.Greys256))[:32], 0, 1
)
data_source['footprint_high'], cm, gl = asphere.add_healpix(footprint_high, nside=nside, cmap=cmap)
data_source['footprint_low'], cm, gl = asphere.add_healpix(footprint_low, nside=low_nside, cmap=cmap)

Create data sources for each band:

In [ ]:
visit_ds = {}
for band in band_palette.keys():
    band_visits = visits.query(f"band=='{band}'")
    if len(band_visits)>0:
        visit_ds[band] = bokeh.models.ColumnDataSource(visits.query(f"band=='{band}'"))
        asphere.connect_controls(visit_ds[band])

Create a transform to hide visits not made until after the date on the MJD slider:

In [ ]:
past_future_js = """
const result = new Array(xs.length)
for (let i = 0; i < xs.length; i++) {
  if (mjd_slider.value >= xs[i]) {
    result[i] = past_value
  } else { 
    result[i] = future_value
  }
}
return result
"""

past_future_transform = bokeh.models.CustomJSTransform(
            args=dict(
                mjd_slider=asphere.sliders['mjd'],
                past_value=0.5,
                future_value=0.0
            ),
            v_func=past_future_js,
        )

In [ ]:
recent_js = """
const result = new Array(xs.length)
for (let i = 0; i < xs.length; i++) {
  if (mjd_slider.value < xs[i]) {
    result[i] = 0
  } else {
    result[i] = Math.max(0, max_value * (1 - (mjd_slider.value - xs[i]) / scale))
  }
}
return result
"""

recent_transform = bokeh.models.CustomJSTransform(
            args=dict(
                mjd_slider=asphere.sliders['mjd'],
                max_value=1.0,
                scale=2.0/(24*60)
            ),
            v_func=recent_js,
        )

Actually add the visits:

In [ ]:
# The visits
for band, band_visit_ds in visit_ds.items():
    asphere.plot.patches(
        xs=asphere.proj_transform('x', band_visit_ds),
        ys=asphere.proj_transform('y', band_visit_ds),
        source=band_visit_ds,
        fill_alpha=bokeh.transform.transform('mjd', past_future_transform),
        line_alpha=bokeh.transform.transform('mjd', recent_transform),
        line_color="black",
        line_width=2,
        fill_color="color",
        legend_label=band
    )

Add standard decorations and guides:

In [ ]:
# The sun
data_source['sun'] = asphere.add_marker(
    sun_coords.ra.deg,
    sun_coords.dec.deg,
    name="Sun",
    glyph_size=15,
    circle_kwargs={"color": "yellow", "legend_label": "Sun"},
)

# The moon
data_source['moon'] = asphere.add_marker(
    moon_coords.ra.deg,
    moon_coords.dec.deg,
    name="Moon",
    glyph_size=15,
    circle_kwargs={"color": "orange", "legend_label": "Moon"},
)

asphere.add_graticules()
asphere.add_ecliptic(color="lightgreen", legend_label="Ecliptic")
asphere.add_galactic_plane(color="lightblue", legend_label="Galactic plane")
data_source['horizon'] = asphere.add_horizon(line_kwargs={"legend_label": "Horizon"})
data_source['zd70'] = asphere.add_horizon(zd=70, line_kwargs={"color": "red", "line_width": 2, "legend_label": "ZD=70" + u'\N{DEGREE SIGN}'})

# Tweak the MJD slider end points to match the first and last visit
asphere.sliders['mjd'].start = visits.mjd.min()
asphere.sliders['mjd'].end = visits.mjd.max()
asphere.sliders['mjd'].value = initial_mjd
asphere.sliders['mjd'].step = 40./(24*60*60)

asphere.plot.add_layout(asphere.plot.legend[0], "right")
# pn.Row(asphere.figure)

Show the planisphere beside the armillary sphere:

In [ ]:
psphere = Planisphere(mjd=initial_mjd)
psphere.sliders['mjd'] = asphere.sliders['mjd']
psphere.add_healpix(data_source['footprint_high'], cmap=cmap)
psphere.add_healpix(data_source['footprint_low'], cmap=cmap)

# The visits
for band, band_visit_ds in visit_ds.items():
    psphere.plot.patches(
        xs=psphere.proj_transform('x', band_visit_ds),
        ys=psphere.proj_transform('y', band_visit_ds),
        source=band_visit_ds,
        fill_alpha=bokeh.transform.transform('mjd', past_future_transform),
        line_alpha=bokeh.transform.transform('mjd', recent_transform),
        line_color="black",
        line_width=2,
        fill_color="color",
    )

psphere.add_marker(
    data_source=data_source['sun'],
    name="Sun",
    glyph_size=15,
    circle_kwargs={"color": "yellow", "legend_label": "Sun"},
)

psphere.add_marker(
    data_source=data_source['moon'],
    name="Moon",
    glyph_size=15,
    circle_kwargs={"color": "orange", "legend_label": "Moon"},
)

psphere.add_graticules()
psphere.add_ecliptic(color="lightgreen")
psphere.add_galactic_plane(color="lightblue")
psphere.add_horizon(data_source=data_source["horizon"])
psphere.add_horizon(
    data_source=data_source["zd70"], line_kwargs={"color": "red", "line_width": 2}
)


viewable = pn.Row(psphere.figure, asphere.figure)

display(viewable)
asphere.update(viewable)
psphere.update(viewable)
pn.io.push_notebook(viewable)

In [ ]:
fname = "night_visits_example.html"

In [ ]:
viewable.save(fname, embed=True)

In [ ]:
with open(fname, 'r', encoding="utf-8") as in_file:
    html_text = in_file.read()

updated_html = html_text.replace(
    "<script",
    """<script src="https://unpkg.com/gpu.js@latest/dist/gpu-browser.min.js"></script>
    <script""",
    1)
    
with open(fname, 'w', encoding='utf-8') as out_file:
    out_file.write(updated_html)